In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import seaborn as sns
from sklearn.metrics import mean_squared_error



In [ ]:
df = pd.read_csv('data/player_stats.csv')
df = df.sort_values(by=['player_id', 'season', 'week'])

## EDA

In [ ]:
df.dtypes 

In [ ]:
df

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
#list all types of data in df
df.dtypes

In [ ]:
sequence_length = 3
tscv = TimeSeriesSplit(n_splits=5)

df = featureEngineering(df)

qb_df_features, qb_df_target, rb_df_features, rb_df_target, wr_df_features, wr_df_target, te_df_features, te_df_target = cleanData(df)

X_qb, y_qb = create_sequences(qb_df_features.values, qb_df_target.values, sequence_length)
X_rb, y_rb = create_sequences(rb_df_features.values, rb_df_target.values, sequence_length)
X_wr, y_wr = create_sequences(wr_df_features.values, wr_df_target.values, sequence_length)
X_te, y_te = create_sequences(te_df_features.values, te_df_target.values, sequence_length)

In [ ]:
X_qb.shape

df

# Prediction

In [ ]:
def plot_predictions(y_true, y_pred, key, split):
    pl.figure(figsize=(10, 6))
    pl.scatter(y_true, y_pred, alpha=0.3)
    pl.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    pl.xlabel('Actual')
    pl.ylabel('Predicted')
    pl.title(f'Predictions vs. Actuals for {key.upper()} - Split {split + 1}')
    pl.grid(True)
    pl.show()


def evaluate_predictions(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'R² Score: {r2}')
    return mse, mae, r2

for key, predictions in rnn_predictions_gru.items():
    for i, prediction in enumerate(predictions):
        dataset = datasets[key][i]
        y_true = dataset['y_test']
        y_pred = prediction

        print(f"Evaluating RNN model for {key} - split {i + 1}...")
        
        # Plot Predictions vs. Actuals
        plot_predictions(y_true, y_pred, key, i)

        # Calculate and print evaluation metrics
        mse, mae, r2 = evaluate_predictions(y_true, y_pred)

        # Optionally, store the metrics for later comparison

evaluation_results = {
    'qb': [],
    'rb': [],
    'wr': [],
    'te': []
}

for key, predictions in rnn_predictions_gru.items():
    for i, prediction in enumerate(predictions):
        dataset = datasets[key][i]
        y_true = dataset['y_test']
        y_pred = prediction

        mse, mae, r2 = evaluate_predictions(y_true, y_pred)
        evaluation_results[key].append((mse, mae, r2))
        
pl.xlabel('Split Number')
pl.ylabel('Mean Squared Error (MSE)')
pl.title('MSE Across Splits for Each Position')
pl.legend()
pl.grid(True)
pl.show()

# Initialize the evaluation results dictionary
evaluation_results = {
    'qb': [],
    'rb': [],
    'wr': [],
    'te': []
}

# Evaluate predictions and store the MSE, MAE, R² score
for key, predictions in rnn_predictions_gru.items():
    for i, prediction in enumerate(predictions):
        dataset = datasets[key][i]
        y_true = dataset['y_test']
        y_pred = prediction

        mse, mae, r2 = evaluate_predictions(y_true, y_pred)
        evaluation_results[key].append(mse)  # Storing only MSE for heatmap

# Convert the evaluation results into a DataFrame for heatmap plotting
import pandas as pd

# Convert evaluation results to a DataFrame
heatmap_data = pd.DataFrame(evaluation_results)

# Plot the heatmap
pl.figure(figsize=(8, 6))
sns.heatmap(heatmap_data.T, annot=True, cmap='coolwarm', fmt=".3f", cbar=True)

pl.xlabel('Split Number')
pl.ylabel('Position')
pl.title('MSE Heatmap Across Splits and Positions')
pl.show()
